# Creation of the CSV I will work on
The first step of the analysis will be to create one main csv from all the csv we got during the scrapping part. From now on we will work on this main_csv. In this file we will also make a first cleaning of the CSV:
* collumns full of 'none' are deleted (Surface_of_the_land)
* lines full of 'none' are deleted
* delete the lines without price (we will compare price in function of the differennt variables) 
* delete the id of the collumn (converting from dataframe to csv we saved the id of the line, we do not need it)
* remove duplicates

In [151]:
import os
import warnings
import pandas as pd

In [152]:
#panda data frame containing every infos
col_names =  ["Locality",     
              "Type_of_property",
              "Subtype_of_property",
              "Price",
              "Type_of_sale",
              "Number_of_rooms",
              "Living_Area",
              "Fully_equipped_kitchen",
              "Furnished",
              "Open_fire",
              "Terrace",
              "Terrace_Area",
              "Garden",
              "Garden_Area",
              "Surface_of_the_land",
              "Surface_area_of_the_plot_of_land",
              "Number_of_facades",
              "Swimming_pool",
              "State_of_the_building"
              ]
df_final  = pd.DataFrame(columns = col_names)

In [153]:
def save_df():
    """
    save tthe global df into a csv
    """
    global df_final
    df_final.to_csv('main_csv.csv', index = False)

In [154]:
def create_df():
    """
    create a dataframe from all the csv we created
    """
    # List all files in the directory
    global df_final
    df_final  = pd.DataFrame(columns = col_names)
    
    #path = "./all_csv"
    path = r"C:\Users\Sacha\Documents\BeCode\real-estate-price-prediction\data_acquisition\all_csv"
    files = os.listdir(path)

    for file in files:
        if os.path.isfile(os.path.join(path, file)):
            file_path = r"C:\Users\Sacha\Documents\BeCode\real-estate-price-prediction\data_acquisition\all_csv\\" + file
            file_df = pd.read_csv(file_path)
            with warnings.catch_warnings():   #warnings from panda because columns with only bool
                warnings.simplefilter("ignore")
                df_final = pd.concat([df_final, file_df])

In [155]:
def first_clean_df():
    """
    this function clean the data a first time
    """
    global df_final
    #remove the useless collumn
    del df_final['Surface_of_the_land']
   
    #drop lines full of nan
    df_final = df_final.dropna(how='all')
    
    #this collumn is the indexes of the dataframe collumn (too much)
    df_final = df_final.drop(df_final.columns[-1],axis=1)
    
    #this line will drop lines where Price is missing
    df_final = df_final.dropna(subset=['Price'])
    
    #remove_duplicates
    df_final = df_final.drop_duplicates()

In [156]:
create_df()
first_clean_df()
save_df()

# Overall analysis

In [157]:
col_names =  ["Locality",     
              "Type_of_property",
              "Subtype_of_property",
              "Price",
              "Type_of_sale",
              "Number_of_rooms",
              "Living_Area",
              "Fully_equipped_kitchen",
              "Furnished",
              "Open_fire",
              "Terrace",
              "Terrace_Area",
              "Garden",
              "Garden_Area",
              "Surface_area_of_the_plot_of_land",
              "Number_of_facades",
              "Swimming_pool",
              "State_of_the_building"
              ]

file_path = r".\main_csv.csv"
df_final = pd.read_csv(file_path)

### number of entries

In [158]:
num_lines = df_final.shape[0]
num_collumns = df_final.shape[1]
print("lines:", num_lines)
print("columns:", num_collumns)

lines: 16647
columns: 18


### unique values

In [159]:
print("numbers of unique values for each columns")
for column in col_names:
    unique = df_final[column].unique()
    if len(unique)>10:
        #print(column, ">10")
        continue
    else:
        print(column, df_final[column].unique())
        print("-------------------------------------")
        
#maison 6 facades???

numbers of unique values for each columns
Type_of_property ['APARTMENT' 'HOUSE']
-------------------------------------
Type_of_sale ['residential_sale' 'first_session_with_reserve_price'
 'annuity_monthly_amount' 'annuity_without_lump_sum']
-------------------------------------
Fully_equipped_kitchen ['USA_HYPER_EQUIPPED' 'NOT_INSTALLED' nan 'INSTALLED' 'HYPER_EQUIPPED'
 'SEMI_EQUIPPED' 'USA_INSTALLED' 'USA_SEMI_EQUIPPED' 'USA_UNINSTALLED']
-------------------------------------
Furnished [False nan True]
-------------------------------------
Open_fire [False  True]
-------------------------------------
Terrace [True nan]
-------------------------------------
Garden [nan True]
-------------------------------------
Number_of_facades [ 4. nan  2.  3.  1.  6.  5.]
-------------------------------------
Swimming_pool [nan False True]
-------------------------------------
State_of_the_building [nan 'AS_NEW' 'JUST_RENOVATED' 'TO_RESTORE' 'GOOD' 'TO_RENOVATE'
 'TO_BE_DONE_UP']
-----------------

We can notice here that there are no more "group_appartment" and "group_house", their price were never given

### % of missing entries

In [149]:
print("percentage of missing values for each columns")
for column in col_names:
    perc =  df_final[column].isnull().sum() / num_lines
    if perc>0:
        print(column, ":", perc)

percentage of missing values for each columns
Price : 0.03629732546022925
Number_of_rooms : 0.03519740650688897
Living_Area : 0.08527266411948593
Fully_equipped_kitchen : 0.2778163714252634
Furnished : 0.22467291883755933
Terrace : 0.33952761375477597
Terrace_Area : 0.5732314461039713
Garden : 0.6317587125159199
Garden_Area : 0.7594072015746208
Surface_area_of_the_plot_of_land : 0.465728841032766
Number_of_facades : 0.27127474817645014
Swimming_pool : 0.45785573694569875
State_of_the_building : 0.2191154335996295


The above values gives us the percentage of missing entries for each columns
* Terrace_area and garden_area have high values, it can be explained because some houses do not have garden/terrace
* Fully_equipped_kitchen, Furnished, Terrace, Garden, Swimming_pool has high values, we could guess that when there is nothing given the value is False, especially for Garden and terrace where the only unique values are True and NaN
* state_of_the_building, Number_of_facade have high values, we could look deeper if it causes problem
* Surface_area_of_the_plot_of_land has a really high value, we should be carrefulll with it after